In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,confusion_matrix, cohen_kappa_score, precision_score
from sklearn.metrics import precision_recall_curve, auc, average_precision_score, f1_score, roc_auc_score, roc_curve
import numpy as np
import pandas as pd
import math
from joblib import dump, load
import random
import pickle
import scipy.stats
from PIL import Image
import scipy.stats as stats

from Functions import post_processing
from Functions import general_functions
from Functions import feature_creation

from skimage.restoration import denoise_bilateral

### Loads all zones with their features and labels

In [2]:
file_list = [f"../zones_features_final/zone_{i}.pickle" for i in range(11, 22)]
importances = np.zeros(0)
predictions = np.zeros(0)

### The following feature set was identified as the optimal in a tuning

In [3]:
features_to_use = ['impundment_mean_3', 'impundment_mean_4', 'impundment_median_4', 'impundment_mean_2',
                   'impundment_amplified', 'hpmf_mean_4', 'impoundment_amplified_no_streams', 'hpmf_median_4',
                   'skyview_max_6', 'skyview_gabor', 'skyview_non_ditch', 'slope_min_6', 'skyview_max_4',
                   'skyview_gabor_no_streams', 'impundment_std_4', 'impundment_max_6', 'slope_non_ditch', 'hpmf_filter',
                   'hpmf_mean_3', 'hpmf_filter_no_streams', 'hpmf_mean_6', 'impundment_median_6', 'impundment_median_2',
                   'slope_std_6', 'impundment_mean_6', 'slope_median_6', 'slope_min_4', 'hpmf_min_2', 'impundment_std_6',
                   'skyview_median_6', 'skyview_min_6', 'slope_mean_6', 'impundment_max_2', 'skyview_std_6', 'slope_min_2',
                   'skyview_max_2', 'hpmf_min_4', 'impundment_max_4', 'slope_std_4', 'hpmf_gabor', 'hpmf_std_6',
                   'slope_max_4', 'hpmf_gabor_no_streams', 'skyview_min_4', 'slope_median_4', 'hpmf_mean_2', 'slope_median_2',
                   'slope_std_2', 'hpmf_min_6', 'hpmf_max_6', 'skyview_mean_6', 'skyview_median_4', 'hpmf_std_4']

### The following parameters were identified as the optimal in a tuning

In [4]:
params = {'criterion': 'gini',
   'max_depth': None,
   'max_features': 'sqrt',
   'min_samples_split': 10,
   'class_weight': None,
   'n_estimators': 300}

### Performs 11 sub-experiments
Trains with 10 zones for each sub-experiment, and evaluates on one zone.
Each zone is evaluated once. Similar to cross validation but without randomly assigned subsets of data.
All of these 11 zones were hold-out data, which means that they had never before been used in our code.

In [5]:
i = 11
for (training_files, test_file) in general_functions.yield_training_test_zones(file_list):
    training_dataset = general_functions.create_balanced_dataset(training_files)
    
    X_train = training_dataset.loc[:, training_dataset.columns != "label_3m"]
    X_train = X_train.filter(items=features_to_use)
    y_train = training_dataset["label_3m"]
    training_dataset = None
    
    clf = RandomForestClassifier(**params, n_jobs=-1)
    clf.fit(X_train,y_train)
    #dump(clf, f"../revised_results/classifier_for_zone_{i}.joblib")
    i += 1
    
    test_dataset = pd.read_pickle(test_file)
    X_test = test_dataset.loc[:, test_dataset.columns != "label_3m"]
    X_test = X_test.filter(items=features_to_use)
    y_test = test_dataset["label_3m"]
    
    proba = clf.predict_proba(X_test)[:,1:].reshape(2997,2620)
    
    proba_post_process = post_processing.proba_post_process(proba, 6, 0.4)
    
    labels_grid = post_processing.raster_to_zones(y_test.reshape(2997, 2620), 6, 4)
    np.append(predictions, (proba, proba_post_process.reshape(-1), labels_grid.reshape(-1)))
    
    np.save("../revised_results/predictions.npy", predictions)
    
    importances = clf.feature_importances_
    feature_names = features_to_use
    tuple_features = [(feature_names[i], importance) for i, importance in enumerate(importances)]
    experiment_importances = np.zeros(len(X_test.shape[1]))
    for f in range(X_test.shape[1]):
        np.append(experiment_importances, (tuple_features[f][0], tuple_features[f][1]*100))
    np.append(importances, experiment_importances)
    np.save("../revised_results/feature_importances_11_experiments.npy", importances)

E:\Skola\Examensarbete\Ditch-article-revision\Functions\post_processing.py:116: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "custom_remove_noise" failed type inference due to: Untyped global name 'd_gf': cannot determine Numba type of <class 'function'>

File "Functions\post_processing.py", line 121:
def custom_remove_noise(arr, radius, threshold, selfThreshold):
    <source elided>
    """
    max_arr = d_gf(da.from_array(arr, chunks=(800, 800)), np.nanmax,
    ^

  @jit
C:\Users\Jonatan\.conda\envs\pytorch\lib\site-packages\numba\core\object_mode_passes.py:177: NumbaWarning: Function "custom_remove_noise" was compiled in object mode without forceobj=True.

File "Functions\post_processing.py", line 117:
@jit
def custom_remove_noise(arr, radius, threshold, selfThreshold):
^

  warnings.warn(errors.NumbaWarning(warn_msg,
C:\Users\Jonatan\.conda\envs\pytorch\lib\site-packages\numba\core\object_mode_passes.py:187: NumbaDeprecationWar

NameError: global name '_create_conic_mask' is not defined

### Loads saved experiment results from file

In [ ]:
predictions = np.load("../revised_results/predictions.npy")
prediction2d = np.concatenate(predictions, axis= 1)
y_pred_all = prediction2d[1]
y_test_all = prediction2d[2]

In [ ]:
predictions = predictions.reshape(11, 3, 2997, 2620)

## Modifying evaluation labels

In [ ]:
for i in range(predictions.shape[0]):
    pred = predictions[i, 1]
    labels = predictions[i, 2]
    new_labels = labels.copy()
    false_positives = np.zeros(pred.shape)
    false_negatives = np.zeros(pred.shape)
    for j in range(pred.shape[0]):
        for k in range(pred.shape[1]):
            if pred[j, k] == 1 and labels[j, k] == 0:
                false_positives[j, k] = 1
            elif pred[j, k] == 0 and labels[j, k] == 1:
                false_negatives[j, k] = 1
    for j in range(0, len(pred) - 6, 6):
        for k in range(0, len(pred[i]) - 6, 6):
            if false_positives[j, k] == 1:
                if labels[j+6, k] == 1 or labels[j-6, k] == 1 or labels[j, k+6] == 1 or labels[j, k-6] == 1:
                    for l in range(j, j+6):
                        for m in range(k, k+6):
                            new_labels[l, m] = 1
            if false_negatives[j, k] == 1:
                if pred[j+6, k] == 1 or pred[j-6, k] == 1 or pred[j, k+6] == 1 or pred[j, k-6] == 1:
                    for l in range(j, j+6):
                        for m in range(k, k+6):
                            new_labels[l, m] = 0
                            
    np.save(f"../revised_results/new_labels_zone_{i+1}.npy", new_labels)

In [ ]:
predictions = np.load("../revised_results/predictions.npy")
prediction2d = np.concatenate(predictions, axis= 1)
y_pred_all = prediction2d[1]
y_test_all = np.load("../revised_results/new_labels_zone_1.npy").reshape(-1)
for i in range (2, 12):
    y_test_all = np.append(y_test_all, np.load(f"../revised_results/new_labels_zone_{i}.npy").reshape(-1))

## Experiment metrics

In [ ]:
print("Accuracy score             ", accuracy_score(y_test_all, y_pred_all))
print("Recall score               ", recall_score(y_test_all, y_pred_all))
print("Precision score            ", precision_score(y_test_all, y_pred_all))
precision, recall, threshholds = precision_recall_curve(y_test_all,y_pred_all)
auc_score = auc(recall, precision)
print("Cohen's kappa score        ", cohen_kappa_score(y_test_all, y_pred_all))
print("AUPRC score                ", auc_score)
print("Confusion matrix\n", confusion_matrix(y_test_all, y_pred_all))

In [ ]:
metrics_per_zone = []
for i, zone in enumerate(predictions):
    proba, binary_prediction, old_labels = zone
    labels_grid = np.load(f"../revised_results/new_labels_zone_{i+1}.npy").reshape(-1)
    acc = accuracy_score(labels_grid, binary_prediction)
    _recall = recall_score(labels_grid, binary_prediction)
    _precision = precision_score(labels_grid, binary_prediction)
    kappa = cohen_kappa_score(labels_grid, binary_prediction)
    precision, recall, threshholds = precision_recall_curve(labels_grid, binary_prediction)
    auc_score = auc(recall, precision)
    metrics = (acc, _recall, _precision, kappa, auc_score)
    metrics_per_zone.append(metrics)
    print(f"Zone {i+1}:\nAccuracy: {metrics[0]}, Recall: {metrics[1]}, Precision: {metrics[2]}, Cohen's Kappa: {metrics[3]}, AUPRC: {metrics[4]}\n")


In [ ]:
def mean_confidence_interval(data, confidence = 0.95):
    sample = np.array(data)
    sample_size, sample_mean = len(sample), sample.mean()
    z_critical = stats.norm.ppf(q = ((1+confidence) / 2.))  # Get the z-critical value*
    margin_of_error = z_critical * (sample.std() / math.sqrt(sample_size))
    return (sample_mean, sample_mean - margin_of_error, sample_mean + margin_of_error)

In [ ]:
metric_names = ["accuracy", "recall", "precision", "Cohen's Kappa", "AUPRC"]
avg = 0
for i in range(0, 5):
    for interval in [0.90, 0.95, 0.99]:
        conf_int = mean_confidence_interval([j[i] for j in metrics_per_zone], interval)
        avg = conf_int[0]
        print(f"Confidence interval for {metric_names[i]} at {interval * 100}%: [{conf_int[1]}, {conf_int[2]}]")
    print(f"Average for {metric_names[i]}: {avg}")
    print("\n")

In [ ]:
plt.figure(num=None, figsize=(15,65), facecolor='w', edgecolor='k')
ax = [plt.subplot(11,3,i+1) for i in range(33)]
for a in ax:
    a.set_xticklabels([])
    a.set_yticklabels([])
    a.tick_params(bottom=False, left=False)
plt.subplots_adjust(wspace=0,hspace=0.1)

for i, zone_pred in enumerate(predictions):
    ax[i*3].title.set_text(f"Raw probability zone {i+1}")
    ax[i*3].imshow(zone_pred[0].reshape(2997,2620))
    ax[i*3+1].title.set_text(f"Binary prediction zone {i+1}")
    ax[i*3+1].imshow(zone_pred[1].reshape(2997,2620))
    ax[i*3+2].title.set_text(f"Labels zone {i+1}")
    ax[i*3+2].imshow(np.load(f"../revised_results/new_labels_zone_{i+1}.npy"))

In [ ]:
plt.figure(num=None, figsize=(30,45), facecolor='w', edgecolor='k')
ax = [plt.subplot(4,3,i+1) for i in range(11)]
for a in ax:
    a.set_xticklabels([])
    a.set_yticklabels([])
    a.tick_params(bottom=False, left=False)
plt.subplots_adjust(wspace=0,hspace=0.1)

for k, zone_pred in enumerate(predictions):
    validation = np.load(f"../revised_results/new_labels_zone_{k+1}.npy")
    pred = zone_pred[1].reshape(2997,2620)
    displayImg = Image.new("RGB", (2620, 2997), "white")
    pixels = displayImg.load()
    for i in range(displayImg.size[0]):
        for j in range(displayImg.size[1]):
            if validation[j][i] == 1 and pred[j][i] == 1:
                pixels[i,j] = (0, 180, 0)
            elif validation[j][i] == 1 and pred[j][i] == 0:
                pixels[i,j] = (50, 150, 255)
            elif validation[j][i] == 0 and pred[j][i] == 1:
                pixels[i,j] = (255, 50, 50)
    
    ax[k].title.set_text(f"Result graphics zone {k+1}")
    ax[k].imshow(displayImg)